In [1]:
# Import required libraries
import pandas as pd
!pip install dash
!pip install dash==1.19.0
!pip install dash dash_core_components dash_html_components jupyter_dash
!pip install --upgrade plotly
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84016 sha256=624419870369a0b4a5cfdcdf746fae2b9ac8d5efcc4b124951ba6643fd449b5e
  Stored in directory: c:\users\rafid\appdata\local\pip\cache\wheels\e7\27\2f\a9f9bf858843b5365bcb311a80f0b2379737caf4ec50b2ae3a
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-py3-none-any.whl size=1014874 sha256=313631ecfa0f1eed169a222b9c086a23c91becf2901c21d6ea122984797b5faf
  Stored in directory: c:\users\rafid\appdata\local\pip\cache\wheels\08\94\09\e11ba8ae5d97e1c8ff5be0ac1f7cf42642eec6ea94ed1e4b44
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527018 sha256=8a0ea0f6f9ea7fe99411dbe346c81e98de04338e23b0dcf8dbdccc52ad09c889
  Stored in directory: c:\users\rafid\appdata\local\pip\cache\wheels\e3\e6\0e\759ca580c568aad2436521fa6df67f53a141dc1ab8f040c4c7
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427836 sha256=b348f

In [10]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

In [11]:
# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})


# List of years 
year_list = [i for i in range(2005, 2021, 1)]

In [12]:
"""Compute graph data for creating yearly airline performance report 

Function that takes airline data as input and create 5 dataframes based on the grouping condition to be used for plottling charts and grphs.

Argument:
     
    df: Filtered dataframe
    
Returns:
   Dataframes to create graph. 
"""
def compute_data_choice_1(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    # Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data




In [13]:
 """Compute graph data for creating yearly airline delay report
 This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    df: Input airline data.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.
"""
def compute_data_choice_2(df):
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


In [14]:
app.layout = html.Div(children=[ 
                                # TODO1: Add title to the dashboard
                                html.H1('US Domestic Airline Flights Performance', 
                                style={'textAlign': 'center', 'color': '#503D36','font-size': 24}),
                            
                                 # REVIEW2: Dropdown creation
                                # Create an outer division 
                                html.Div([
                                    # Add an division
                                     html.Div([
                                        # Create an division for adding dropdown helper text for report type
                                        html.Div(
                                            [
                                            html.H2('Report Type:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        # TODO2: Add a dropdown
                                        dcc.Dropdown(
                                        id='input-type',
                                        options=[
                                            {'label': 'Yearly Airline Performance Report', 'value': 'OPT1'},
                                            {'label': 'Yearly Airline Delay Report', 'value': 'OPT2'}
                                        ],
                                        placeholder='Select a report type',
                                        style={'width':'80%','padding':'3px','font size':'20px','text-align-last':'center'})
                               
                                    # Place them next to each other using the division style
                                    ], style={'display':'flex'}),
                                    
                                   # Add next division 
                                   html.Div([
                                       # Create an division for adding dropdown helper text for choosing year
                                        html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-year', 
                                                     # Update dropdown values using list comphrehension
                                                     options=[{'label': i, 'value': i} for i in year_list],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            # Place them next to each other using the division style
                                            ], style={'display': 'flex'}),  
                                          ]),
                                
                                # Add Computed graphs
                                # REVIEW3: Observe how we add an empty division and providing an id that will be updated during callback
                                html.Div([ ], id='plot1'),
    
                                html.Div([
                                        html.Div([ ], id='plot2'),
                                        html.Div([ ], id='plot3')
                                ], style={'display': 'flex'}),
                                
                                # TODO3: Add a division with two empty divisions inside. See above disvision for example.
                                html.Div([
                                        html.Div([ ], id='plot4'),
                                        html.Div([ ], id='plot5')
                                ], style={'display': 'flex'}),
                                ])

# Callback function definition
# TODO4: Add 5 ouput components\'[p]'

@app.callback( [Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
                Output(component_id='plot3', component_property='children'),
                Output(component_id='plot4', component_property='children'),
                Output(component_id='plot5', component_property='children')
               ],
               [Input(component_id='input-type', component_property='value'),
                Input(component_id='input-year', component_property='value')],
               # REVIEW4: Holding output state till user enters all the form information. In this case, it will be chart type and year
               [State("plot1", 'children'), State("plot2", "children"),
                State("plot3", "children"), State("plot4", "children"),
                State("plot5", "children")
               ])
# Add computation to callback function and return graph
def get_graph(chart, year, children1, children2, c3, c4, c5):
      
        # Select data
        df =  airline_data[airline_data['Year']==int(year)]
       
        if chart == 'OPT1':
            # Compute required information for creating graph from the data
            bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df)
            
            # Number of flights under different cancellation categories
            bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
            
            # TODO5: Average flight time by reporting airline
            line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline', title='Average monthly flight time (minutes) by airline')
            
            # Percentage of diverted airport landings per reporting airline
            pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
            
            # REVIEW5: Number of flights flying from each state using choropleth
            map_fig = px.choropleth(map_data,  # Input data
                    locations='OriginState', 
                    color='Flights',  
                    hover_data=['OriginState', 'Flights'], 
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale='GnBu',
                    range_color=[0, map_data['Flights'].max()]) 
            map_fig.update_layout(
                    title_text = 'Number of flights from origin state', 
                    geo_scope='usa') # Plot only the USA instead of globe
            
            # TODO6: Number of flights flying to each state from each reporting airline
            
            tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'], 
                      values='Flights',
                      color='Flights',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state')               
            
            # REVIEW6: Return dcc.Graph component to the empty division
            return [dcc.Graph(figure=tree_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=map_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)
                   ]
        else:
            # REVIEW7: This covers chart type 2 and we have completed this exercise under Flight Delay Time Statistics Dashboard section
            # Compute required information for creating graph from the data
            avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
            
            # Create graph
            carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
            weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
            nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
            sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
            late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
            return[dcc.Graph(figure=carrier_fig), 
                   dcc.Graph(figure=weather_fig), 
                   dcc.Graph(figure=nas_fig), 
                   dcc.Graph(figure=sec_fig), 
                   dcc.Graph(figure=late_fig)]

In [15]:
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

127.0.0.1 - - [27/Apr/2021 10:29:36] "GET /_shutdown_00ca28a7-6b5e-4ed4-8f12-be0a00a0fac5 HTTP/1.1" 200 -
 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2021 10:29:38] "GET /_alive_00ca28a7-6b5e-4ed4-8f12-be0a00a0fac5 HTTP/1.1" 200 -


127.0.0.1 - - [27/Apr/2021 10:29:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:29:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:29:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:29:39] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-2c9d162eb6de> in get_graph(
    chart=None,
    year=None,
    children1=[],
    children2=[],
    c3=[],
    c4=[],
    c5=[]
)
     81 
     82         # Select data
---> 83         df =  airline_data[airline_data['Year']==int(year)]
        df = undefined
        global airline_data =        Unnamed: 0  Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0         1295781  1998        2      4           2          4  1998-04-02   
1         1125375  2013        2      5          13          1  2013-05-13   
2          118824  1993        3      9          25          6  1993-09-25   
3          634825  1994        4     11          12          6  1994-11-12   
4         1888125  2017        3      8          17          4  2017-08-17   
...           ...   ...      ...    ...         ...        ...         ..

127.0.0.1 - - [27/Apr/2021 10:29:48] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-2c9d162eb6de> in get_graph(
    chart='OPT1',
    year=None,
    children1=[],
    children2=[],
    c3=[],
    c4=[],
    c5=[]
)
     81 
     82         # Select data
---> 83         df =  airline_data[airline_data['Year']==int(year)]
        df = undefined
        global airline_data =        Unnamed: 0  Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0         1295781  1998        2      4           2          4  1998-04-02   
1         1125375  2013        2      5          13          1  2013-05-13   
2          118824  1993        3      9          25          6  1993-09-25   
3          634825  1994        4     11          12          6  1994-11-12   
4         1888125  2017        3      8          17          4  2017-08-17   
...           ...   ...      ...    ...         ...        ...         

127.0.0.1 - - [27/Apr/2021 10:29:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:29:53] "GET /_dash-component-suites/dash_core_components/async-graph.v1_15_0m1611086576.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:29:53] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_15_0m1611086576.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:30:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:30:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:30:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:30:27] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:30:28] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-2c9d162eb6de> in get_graph(
    chart=None,
    year=None,
    children1=[],
    children2=[],
    c3=[],
    c4=[],
    c5=[]
)
     81 
     82         # Select data
---> 83         df =  airline_data[airline_data['Year']==int(year)]
        df = undefined
        global airline_data =        Unnamed: 0  Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0         1295781  1998        2      4           2          4  1998-04-02   
1         1125375  2013        2      5          13          1  2013-05-13   
2          118824  1993        3      9          25          6  1993-09-25   
3          634825  1994        4     11          12          6  1994-11-12   
4         1888125  2017        3      8          17          4  2017-08-17   
...           ...   ...      ...    ...         ...        ...         ..

127.0.0.1 - - [27/Apr/2021 10:46:46] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-14-2c9d162eb6de> in get_graph(
    chart='OPT1',
    year=None,
    children1=[],
    children2=[],
    c3=[],
    c4=[],
    c5=[]
)
     81 
     82         # Select data
---> 83         df =  airline_data[airline_data['Year']==int(year)]
        df = undefined
        global airline_data =        Unnamed: 0  Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0         1295781  1998        2      4           2          4  1998-04-02   
1         1125375  2013        2      5          13          1  2013-05-13   
2          118824  1993        3      9          25          6  1993-09-25   
3          634825  1994        4     11          12          6  1994-11-12   
4         1888125  2017        3      8          17          4  2017-08-17   
...           ...   ...      ...    ...         ...        ...         

127.0.0.1 - - [27/Apr/2021 10:46:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:49:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:52:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:52:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:52:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:55:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:55:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 10:56:32] "POST /_dash-update-component HTTP/1.1" 200 -
